In [1]:
from datasets import load_from_disk
import pandas as pd
import json
from pprint import pprint
import re
import random
import numpy as np
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

def dataset_to_df(dataset):
    return pd.DataFrame(dataset)

In [3]:
path = "" # 넣어줄 파일  지정
dataset = load_from_disk(path)

In [4]:
train_df = dataset_to_df(dataset["train"]) 

In [ ]:
train_df.head()

In [6]:
print(len(train_df))

3952


In [7]:
random.seed(42)


# 사용할 수 있는 구두점 정의
PUNCTUATIONS = [",", ".", "!", "?"]

# 구두점을 텍스트에 적절히 삽입하는 함수
def insert_punctuation_marks(text, answer_start, answer_length):
    words = text.split()
    answer_end = answer_start + answer_length
    # 정답에 포함된 단어의 인덱스
    qs = [i for i in range(len(words)) if answer_start <= sum(len(w) + 1 for w in words[:i]) < answer_end]

    new_line = []
    for j, word in enumerate(words):
        # 각 단어의 시작 및 끝 인덱스 계산
        word_start = sum(len(w) + 1 for w in words[:j])
        word_end = word_start + len(word)

        # 정답 텍스트 범위 내에는 구두점을 삽입하지 않음
        if j not in qs and not (word_start >= answer_start and word_end <= answer_end):
            new_line.append(PUNCTUATIONS[random.randint(0, len(PUNCTUATIONS)-1)])
            new_line.append(word)
        else:
            new_line.append(word)
    
    # 단어와 구두점을 하나의 문자열로 결합
    new_line = ' '.join(new_line)
    return new_line

# AEDA를 적용하는 함수
def apply_aeda(df, text_column, num_augs=1):
    augmented_data = []
    
    for _, row in df.iterrows():
        original_text = row[text_column]
        # 정답의 올바른 인덱스 추출
        answer_start = row['answers']['answer_start'][0]
        answer_length = len(row['answers']['text'][0])
        
        # 원본 데이터를 증강 데이터 목록에 추가
        augmented_data.append(row.to_dict())
        
        for _ in range(num_augs):
            # 구두점 삽입을 통해 증강된 텍스트 생성
            augmented_text = insert_punctuation_marks(original_text, answer_start, answer_length)
            # 증강된 텍스트를 포함한 행의 복사본 생성
            augmented_row = row.to_dict()
            augmented_row[text_column] = augmented_text
            # 증강된 행을 목록에 추가
            augmented_data.append(augmented_row)

    # 원본 및 증강된 데이터를 모두 포함하는 새로운 DataFrame 반환
    return pd.DataFrame(augmented_data)

# 예시 데이터프레임으로 AEDA 적용 예시

train_dft = pd.DataFrame(train_df)

# AEDA 적용
df_augmented = apply_aeda(train_df, 'context', num_augs=1)

In [9]:
#위치를 맞추기 위해 데이터를 테스트
df_augmented_test = df_augmented.copy()

In [10]:
# context와 같은 단어들의 시작 인덱스를 구하는 코드
def get_answer_word_idx(context, answer):
  indices = [m.start() for m in re.finditer(answer, context)]
  return indices

In [11]:
df_augmented_test["answer_word_idx"] = df_augmented_test.apply(
    lambda row: get_answer_word_idx(row["context"], row["answers"]["text"][0]), axis=1
)

In [12]:
# 생성한 answer_word_idx에서 answer_start와 같은 값을 가지는 인덱스 번호 저장
def get_answer_idx(answer_start, answer_word_idx):
  re_idx = 0
  for idx, data in enumerate(answer_word_idx):
    if data == answer_start:
      re_idx = int(idx)

  return re_idx

In [17]:
df_augmented_test_cleaned = df_augmented_test[df_augmented_test['answer_word_idx'].apply(lambda x: len(x) > 0)] # answer_word_idx 위치를 찾지 못한 데이터 제거

In [19]:
empty_answer_word_idx = df_augmented_test[df_augmented_test['answer_word_idx'].apply(lambda x: len(x) == 0)] #answer_word_idx 위치를 찾지 못한 데이터 확인

In [22]:
# 증강된 데이터만 선택 (홀수 인덱스)
augmented_data_mask = df_augmented_test.index % 2 != 0
df_augmented_agmented = df_augmented_test[augmented_data_mask].reset_index(drop=True)

In [24]:
# 증강안 된 데이터만 선택 (짝수 인덱스)
augmented_data_mask1 = df_augmented_test.index % 2 == 0
df_augmented_agmented33 = df_augmented_test[augmented_data_mask1].reset_index(drop=True)

In [ ]:
# 올바른 answer_start위치 찾기
def align_answer_start_with_aeda(df_orig, df_augmented):
    for index, row in df_augmented.iterrows():
        # 원본 데이터의 answer_word_idx와 answer_start 가져오기
        original_word_indices = df_orig.at[index, 'answer_word_idx']
        original_answer_start = df_orig.at[index, 'answers']['answer_start'][0]
        
        # 증강된 데이터의 answer_word_idx 가져오기
        augmented_word_indices = row['answer_word_idx']

        if original_word_indices and augmented_word_indices:
            # 원본 데이터에서 answer_start의 인덱스 찾기
            original_index = original_word_indices.index(original_answer_start)
            
            # 증강된 데이터에서 같은 인덱스의 값 찾기
            if original_index < len(augmented_word_indices):
                aligned_index = augmented_word_indices[original_index]
                
                # 기존 answers 딕셔너리에서 answer_start 값을 업데이트
                updated_answers = row['answers'].copy()
                updated_answers['answer_start'] = [aligned_index]
                df_augmented.at[index, 'answers'] = updated_answers
                print(f"Augmented Row {index}: Aligned answer_start to {aligned_index}")
            else:
                print(f"Augmented Row {index}: Index out of range")
        else:
            print(f"Row {index}: No valid indices found")

# 함수 실행
align_answer_start_with_aeda(df_augmented_agmented33, df_augmented_agmented)

In [29]:
df_augmented_cle_cleaned = df_augmented_agmented[df_augmented_agmented['answer_word_idx'].apply(lambda x: len(x) > 0)]
print(len(df_augmented_cle_cleaned))

3826


In [ ]:
df_augmented_cle_cleaned.drop('answer_word_idx', axis=1, inplace=True)
df_augmented_agmented33.drop('answer_word_idx', axis=1, inplace=True)
# 필요하지 않은 컬럼 삭제

In [31]:
df_half = df_augmented_cle_cleaned.copy() # 반절의 데이터 활용을 위한 데이터 복사

In [33]:
df_combined = pd.concat([df_augmented_agmented33, df_augmented_cle_cleaned], ignore_index=True) # 나눈 데이터에 대한 병합 

In [34]:
df_combined.to_csv('/data/ephemeral/home/lji/orginal.csv', index=False) # 원본과 전체 증강 데이터

In [36]:
np.random.seed(42)  
df_half = df_half.sample(frac=0.5) # 랜덤 시드를 전해준 후 값 추출

In [38]:
df_combined2 = pd.concat([df_augmented_agmented33, df_half], ignore_index=True) #원본 + 1/2 증강 데이터

In [39]:
df_combined2.to_csv('/data/ephemeral/home/lji/half2.csv', index=False) # 원본 + 1/2 증강 데이터

In [ ]:
dataset = load_dataset('csv', data_files='/data/ephemeral/home/lji/orginal.csv')
dataset_half = load_dataset('csv', data_files='/data/ephemeral/home/lji/half2.csv') # 허깅페이스 형태로 


In [41]:
print(dataset,dataset_half)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'document_id', 'answers'],
        num_rows: 7778
    })
}) DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'document_id', 'answers'],
        num_rows: 5865
    })
})


In [41]:
# Arrow 포맷으로 저장
dataset.save_to_disk('/data/ephemeral/home/lji') # 파일 저장 

Saving the dataset (1/1 shards): 100%|██████████| 7778/7778 [00:00<00:00, 304327.47 examples/s]


In [ ]:
dataset_half.save_to_disk('/data/ephemeral/home/lji')

In [ ]:
#그 외 문장 확인 코드
def verify_answer_indices(df):
    for index, row in df.iterrows():
        context = row['context']
        answer_text = row['answers']['text'][0]
        expected_start = row['answers']['answer_start'][0]
        word_indices = row['answer_word_idx']

        # Check if the expected start index is in the list of word indices
        if expected_start in word_indices:
            print(f"Row {index}: Correct - '{answer_text}' found at {expected_start}")
        else:
            print(f"Row {index}: Incorrect - '{answer_text}' not found at expected index {expected_start}")
            # Optionally print the context and indices for debugging
            print(f"Context: {context}")
            print(f"Word Indices: {word_indices}")

# Use the function on your augmented DataFrame
verify_answer_indices(df_augmented_agmented)